<a href="https://colab.research.google.com/github/anik314159/Hate-Alert-Urdu-Sentiment-Analysis-project-/blob/main/Urdu_Hate_Alert_Task_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import torch 
import torch.nn.functional as F
import torch.nn as nn
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')


In [ ]:
train_Data = pd.read_csv('/content/drive/MyDrive/urdu_hate_task_B/Training Dataset.csv')

In [ ]:
train_Data

,Tweets,label,S/G
0,بکواس مت کرو,1,1
1,تمہاری ہیجڑا فورس ایک نہتے کے سامنے بکری بنی ہ...,1,0
2,آفیسر سمیتبھارتی فوجی جہنم واصل،بنکرز تباہ بھا...,1,1
3,غدار منافق میر اللہ تمہیں زلیل کرے,1,1
4,اگست پر آپ بھارت کو کیا پیغام دینا چاہیں گے؟ م...,1,0
...,...,...,...
3559,ارے شعر ہے حقیقت میں کہیں پیچھے نا پڑنا ۔,0,2
3560,ہندوستانی باوردی غنڈا ایل او سی پر پتھر مار کے...,0,2
3561,یہ خبر افسوس ناک ھے,0,2
3562,جناب محترم سلمان وفادارصاھب کیا پارلیمنٹ آئینی...,0,2


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 33.3 MB/s 
     |████████████████████████████████| 6.6 MB 52.6 MB/s 
     |████████████████████████████████| 120 kB 73.8 MB/s 


In [ ]:
from transformers import BertTokenizer
Paths_to_transformers = { 'muril': 'google/muril-base-cased',
                         'mbert': 'bert-base-multilingual-cased'}
tokenizer = BertTokenizer.from_pretrained(Paths_to_transformers['mbert'])

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
  def __init__(self,df,max_len,tokenizer):
    self.df = df
    self.max_len = max_len
    self.tokenizer = tokenizer
    self.label = self.df['label']
    self.title = self.df['Tweets']

  def __len__(self):
    return len(self.title)

  def __getitem__(self,index):
    text = self.title.iloc[index]
    text = " ".join(text.split())
    inputs = self.tokenizer.encode_plus(text,add_special_tokens = True,
                                  max_length=self.max_len,
                                  padding = 'max_length',
                                  return_token_type_ids = True,
                                  truncation = True,
                                  return_attention_mask = True,
                                  return_tensors = 'pt')
    
    return {
        'input_ids' : inputs['input_ids'].flatten(),
        'attention_mask': inputs['attention_mask'].flatten(),
        'token_type_ids': inputs['token_type_ids'].flatten(),
        'labels': torch.FloatTensor([float(self.label.iloc[index]),abs(1-self.label.iloc[index])])
    }





In [ ]:
train_df , val_df = train_test_split(train_Data,test_size=0.2,random_state=2020)

train_dataset = CustomDataset(train_df,128,tokenizer)
val_dataset   = CustomDataset(val_df  ,128,tokenizer)

In [ ]:
train_df


,Tweets,label,S/G
2177,بھارت کے ایٹمی ہتھیار فاشسٹ مودی حکومت کے ہاتھ...,0,2
95,انشاءاللہ کتے کی موت مریں گے۔,1,0
3286,سرینگر بھارتی فوج کی دہشت گردی کا سلسلہ جاری۔,0,2
3000,دل خون کے آنسو رو رہا ہے۔ مسلمان امہ کی بے بسی...,0,2
2390,ان بےشرم بےغیرتوں کو دیکھو کھانا کیسے ٹھونس رہ...,0,2
...,...,...,...
2139,فائیو جی ٹیکنالوجی خطرناک ہے؟,0,2
2678,کچھ قصاٸی صرف بکرے کی نہیں ہماری کھال اتار لین...,0,2
3395,آرمی چیف کو توسیع دینے سے ایک بات ضرور ثابت ہو...,0,2
392,اک بات بتاؤ اگر کویی تمہاری گانڈ مار دے تو تم ...,1,0


In [ ]:
from torch.utils.data import DataLoader
train_data_loader = torch.utils.data.DataLoader( train_dataset , shuffle = True  , batch_size = 16, num_workers = 2 )
val_data_loader = torch.utils.data.DataLoader(val_dataset , shuffle =True , batch_size = 16, num_workers = 2)

from transformers import BertConfig,BertForSequenceClassification
from transformers import *

In [ ]:
from transformers import BertModel 
class BertClass(BertPreTrainedModel):
  def __init__(self,config,weights):
    super().__init__(config)
    self.num_classes = config.num_labels
    self.weights = weights
    self.bert_model = BertModel(config)
    self.dropout = nn.Dropout(config.hidden_dropout_prob)
    self.linear = nn.Linear(config.hidden_size, self.config.num_labels,dtype = torch.float)
    self.loss_fn = nn.CrossEntropyLoss(weight = torch.tensor(weights),label_smoothing = 0.0)
    self.init_weights()
    
  def forward(self, input_ids=None,attention_mask=None,token_type_ids=None,position_ids=None, head_mask=None,inputs_embeds=None, labels=None):
    outputs = self.bert_model(input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids,
                             position_ids=position_ids,head_mask=head_mask,inputs_embeds=inputs_embeds)


    pooled_output = outputs[1]

    pooled_output = self.dropout(pooled_output)
    logits = self.linear(pooled_output)

    outputs = logits 
    #(logits,) + outputs[2:]  # add hidden states and attention if they are here

    #if labels is not None:
     
    # loss = self.loss_fn((logits).float(), (labels).float())
      
    #outputs = (loss,) + outputs

    return outputs  # (loss), logits, (hidden_states), (attentions)


 
    

class_weights = torch.Tensor([train_df['label'].sum()/len(train_df['label']) , 1 - train_df['label'].sum()/len(train_df['label']) ]).to(device)
print(class_weights)

model = BertClass.from_pretrained(Paths_to_transformers['mbert'],num_labels = 2,output_attentions =False ,output_hidden_states = False,weights = class_weights)
model.cuda()

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/cf732291d5a8eace7b973ccd13c95ec07b19e734/config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.22.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 119547
}



tensor([0.4946, 0.5054], device='cuda:0')


Downloading:   0%|          | 0.00/714M [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/cf732291d5a8eace7b973ccd13c95ec07b19e734/pytorch_model.bin
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertClass: ['bert.encoder.layer.1.attention.output.LayerNorm.bias', 'bert.encoder.layer.11.output.dense.bias', 'bert.encoder.layer.0.attention.self.key.weight', 'bert.encoder.layer.6.output.LayerNorm.weight', 'bert.pooler.dense.bias', 'bert.encoder.layer.7.attention.output.LayerNorm.bias', 'bert.encoder.layer.6.attention.output.LayerNorm.weight', 'bert.encoder.layer.9.attention.output.dense.we

BertClass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=T

In [ ]:


loss_model = nn.CrossEntropyLoss(weight = torch.Tensor(class_weights))
optimizer = torch.optim.Adam(params = model.parameters() , lr = 2e-5 ,eps = 1e-8)




In [ ]:
import gc


gc.collect()
from transformers import get_linear_schedule_with_warmup
bestvalacc = 0
loss_val = []

def train_model(n_epoch , train_loader , val_loader , model,optimizer):
  val_loss_min = np.inf
  for epoch in range(1,n_epoch+1):
    #torch.cuda.empty_cache()
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = len(train_data_loader)* n_epoch)
    train_loss = 0;
    valid_loss = 0;
    valid_acc = 0;
    nb_eval_steps = 0;
    model.train();
    print(" >>>>>>>>> Epoch {} : Training..".format(epoch))

    for batch_idx, data in enumerate(train_loader):
      
      if batch_idx % 40 == 0  and not batch_idx == 0:
        print('Batch : {} / {}'.format(batch_idx,len(train_loader)))

      ids = data['input_ids'].to(device , dtype = torch.long)
      mask = data['attention_mask'].to(device,dtype = torch.long)
      token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
      targets = data['labels'].to(device, dtype = torch.float)
      

      outputs = model(ids, attention_mask = mask, token_type_ids = token_type_ids,labels = targets)
      outputs = torch.sigmoid(outputs).to(device,dtype = torch.float)

      #outputs = torch.Tensor([float(ele[0] > ele[1]) for ele in outputs])
      #print(outputs)
      #outputs = torch.transpose(outputs,0,1)
      #print(outputs.flatten())
      #print(outputs)
      loss = loss_model(outputs,targets)
      #loss = outputs[0]
      
      optimizer.zero_grad()
      loss.backward()      
      train_loss += loss.item()
      torch.nn.utils.clip_grad_norm(model.parameters(),1.0)
      optimizer.step()
      scheduler.step();

    avg_train_loss =  train_loss / len(train_loader)
    loss_val.append(avg_train_loss)

    print(" >>>>>>> Epoch {} / {}: Training end >...< average loss: {} ".format(epoch , n_epoch, avg_train_loss))

    print(" >>>>>>> Running validation ")

    model.eval();

    val_loss = []
    val_accuracy = []
    bestvalacc =0;
    for data in val_loader:
      in_id = data['input_ids'].to(device,dtype = torch.long)    
      at_mask = data['attention_mask'].to(device,dtype = torch.long)
      t_t_id = data['token_type_ids'].to(device,dtype = torch.long)
      targets = data['labels'].to(device,dtype = torch.float)
     
      
      with torch.no_grad():
        outputs = model(in_id,attention_mask = at_mask,token_type_ids =t_t_id)
        outputs = torch.sigmoid(outputs)
      loss = loss_model(outputs,targets)
      #loss = outputs.flatten()
      logits = outputs     
      val_loss.append(loss.item())
    

      logits = logits.detach().cpu().numpy()
      label_ids = targets.cpu().numpy()

      temp_accuracy  = np.sum(logits.flatten() == label_ids.flatten()) / len(label_ids.flatten()) 
      val_accuracy.append(temp_accuracy)

    val_loss = np.mean(val_loss)
    print(" >>>>>>> Epoch {} / {}: validation end >...< average loss: {} ".format(epoch , n_epoch, val_loss))
    val_accuracy = np.mean(val_accuracy)
    
    if val_loss < val_loss_min:
      print("minimum Validation updated from {} to {}".format(val_loss_min,val_loss))
      val_loss_min = val_loss
      torch.save(model,"/content/drive/MyDrive/urdu_hate_task_B/mbert_model_best.pth")
      

    print(">>>>>>Epoch {} completed".format(epoch)) 
  return model 
trained_model = train_model(10,train_data_loader,val_data_loader,model,optimizer)

 >>>>>>>>> Epoch 1 : Training..


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


Batch : 40 / 179
Batch : 80 / 179
Batch : 120 / 179
Batch : 160 / 179
 >>>>>>> Epoch 1 / 10: Training end >...< average loss: 0.3466915228513366 
 >>>>>>> Running validation 
 >>>>>>> Epoch 1 / 10: validation end >...< average loss: 0.34651412434048123 
minimum Validation updated from inf to 0.34651412434048123
>>>>>>Epoch 1 completed
 >>>>>>>>> Epoch 2 : Training..
Batch : 40 / 179
Batch : 80 / 179
Batch : 120 / 179
Batch : 160 / 179
 >>>>>>> Epoch 2 / 10: Training end >...< average loss: 0.3465852525980113 
 >>>>>>> Running validation 
 >>>>>>> Epoch 2 / 10: validation end >...< average loss: 0.34650387631522284 
minimum Validation updated from 0.34651412434048123 to 0.34650387631522284
>>>>>>Epoch 2 completed
 >>>>>>>>> Epoch 3 : Training..
Batch : 40 / 179
Batch : 80 / 179
Batch : 120 / 179
Batch : 160 / 179
 >>>>>>> Epoch 3 / 10: Training end >...< average loss: 0.34665504547470777 
 >>>>>>> Running validation 
 >>>>>>> Epoch 3 / 10: validation end >...< average loss: 0.3464177389

KeyboardInterrupt: ignored

In [ ]:
import matplotlib.pyplot as plt


plt.plot(loss_val,'b-o')
plt.title("Training loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.show()

In [ ]:
#model = torch.load('/content/drive/MyDrive/urdu_hate_task_B/muril_model_best.pth')

class CustomDataset_test(torch.utils.data.Dataset):
  def __init__(self,df,tokenizer,max_len):
    self.df = df;
    self.tokenizer = tokenizer
    self.max_len = max_len
    self.title = self.df['Tweet']
    #self.targets = self.df[target_list].values

  def __len__(self):
    return len(self.title)

  def __getitem__(self,index):
    text = (self.title.iloc[index])
    
    text = " ".join(text.split()) #return to this function 

    inputs = self.tokenizer.encode_plus(text,add_special_tokens = True,
                                  max_length=self.max_len,
                                  padding = 'max_length',
                                  return_token_type_ids = True,
                                  truncation = True,
                                  return_attention_mask = True,
                                  return_tensors = 'pt')
    
    return {
        'input_ids' : inputs['input_ids'].flatten(),
        'attention_mask' : inputs['attention_mask'].flatten(),
        'token_type_ids' : inputs['token_type_ids'].flatten()        
    }


In [ ]:
test_Data = pd.read_csv("/content/drive/MyDrive/urdu_hate_task_B/test_data.csv")
test_Data

In [ ]:


model.eval()
predictions , true_labels = [] , []
tokenizer =  BertTokenizer.from_pretrained(Paths_to_transformers['mbert'])
test_dataset = CustomDataset_test(test_Data,tokenizer,128)

test_data_loader = torch.utils.data.DataLoader( test_dataset , shuffle = True  , batch_size = 16,num_workers = 2 )


for data in (test_data_loader):
  in_id = data['input_ids'].to(device,dtype = torch.long)
  at_mask = data['attention_mask'].to(device,dtype = torch.long)
  t_t_id = data['token_type_ids'].to(device,dtype = torch.long)

  with torch.no_grad():
    outputs = model(in_id,at_mask,t_t_id)
    #print(outputs)
  logits = outputs
  
  logits = torch.from_numpy(logits.detach().cpu().numpy())
  tensor = torch.nn.functional.softmax(logits,dim=-1)
  print(tensor)
  for tens in tensor:
    #tens = [ int(prob >= 0.5) for prob in tens ]
    tens = np.argmax(tens.numpy())
    predictions.append(tens)
    


  

print('DONE')

In [ ]:
for i in range(len(test_Data)):
  test_Data['Threat'].iloc[i] = predictions[i]




In [ ]:
test_Data

In [ ]:
from pathlib import Path  

filepath = Path('/content/drive/MyDrive/urdu_hate_task_B/out_taskB_pt1_mbert.csv')
filepath.parent.mkdir(parents=True, exist_ok=True,)  

test_Data.to_csv(filepath)

In [ ]:
filepath = Path('/content/drive/MyDrive/urdu_hate_task_B/hate-alert_Run1_TaskB_mbert.zip')
filepath.parent.mkdir(parents=True, exist_ok=True,)  
compression_opts = dict(method='zip',archive_name='out_taskB_pt1_mbert.csv') 
test_Data.to_csv(filepath,compression = compression_opts)

In [ ]:
test_Data